In [27]:
import os
import sys
import logging
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv

load_dotenv()

# API 키 설정
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    logging.error("API 키를 설정해주세요.")

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)

In [28]:
# === 문서 로드 및 인덱스 생성 (RAG) ===
# 문서 폴더 내에는 아래 데이터가 포함되어 있다고 가정.
# 1. 합격 자소서 데이터, 2. 합격자 면접 데이터, 3. 업계 동향 데이터, 4. 업계 구인 데이터

# 문서 로드를 위한 패키지
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Document
from llama_index.core import GPTVectorStoreIndex

# 문서 로드 및 인덱스 생성
data_path = os.getenv("DATA_PATH")
documents = SimpleDirectoryReader(data_path).load_data()
if not data_path:
    logging.error("데이터 경로를 설정해주세요.")

index = GPTVectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [29]:
# 참고 자료 검색: 자소서 분석 및 피드백에 유용한 자료를 외부 문서에서 추출
reference_query = (
    "합격 자소서, 합격자 면접, 업계 동향, 구인 데이터를 참고하여, "
    "자소서 작성 피드백에 유용한 핵심 인사이트를 간략히 요약해줘."
)
reference_result = query_engine.query(reference_query)

# 지원 기업 관련 산업 뉴스 데이터 추출 (지원 기업 및 관련 키워드 활용)
company = "A기업"
industry_news_query = (
    f"{company} 관련 산업 뉴스, 최신 동향, 전략 및 이슈"
)
industry_news_result = query_engine.query(industry_news_query)


# 두 결과를 결합하여 참고 자료로 활용
combined_reference = f"{reference_result}\n\n[산업 뉴스 참고]:\n{industry_news_result}"


In [30]:
## 여기부터 실행

# === 자소서 분석 체인 프롬프트 템플릿 ===
init_prompt = PromptTemplate(
    input_variables=["참고자료"],
    template="""
    이전 대화 내용을 무시하고, 아래에 제공된 정보만을 기반으로 분석을 수행해 주세요.
    이후에 입력하는 내용을 기반으로 자소서 분석, 조언, 피드백을 제공해주세요.
    --------------------------------------
    {참고자료} 
    --------------------------------------
    """
)

In [31]:
# === 수정된 분석 체인 프롬프트 템플릿 (회사, 직무 정보 포함 및 적합성 체크) ===
analysis_prompt = PromptTemplate(
    input_variables=["자소서내용", "참고자료", "회사", "직무"],
    template="""
사용자가 입력한 자소서 내용은 다음과 같습니다:
--------------------------------------
{자소서내용}
--------------------------------------
아래의 참고 자료는 합격 자소서, 면접 데이터, 업계 동향, 구인 데이터 및 지원 기업 관련 산업 뉴스에서 추출된 인사이트입니다:
--------------------------------------
{참고자료}
--------------------------------------
또한, 사용자가 지원하는 회사는 "{회사}"이며 희망 직무는 "{직무}"입니다.
위 정보를 종합하여, 다음 사항들을 분석하고, 예상 합격/불합격 여부를 알려주고 점수도 내줘(예 : 8.0/10 ).:
자소서에 나타난 강점과 약점을 각각 3가지씩 구체적으로 제시
자소서 내용이 지원하는 회사와 희망 직무에 얼마나 부합하는지 평가하고, 개선이 필요한 부분(예: 지원 동기, 관련 경험, 역량 표현 등)을 구체적으로 지적해줘.
예시) 강점: [문제 해결 능력, 팀워크 등] / 약점: [경험 부족, 구체성 부족 등] / 적합성 평가: [지원 동기 보완 필요 등]
"""
)

In [32]:
# === 조언 체인 프롬프트 템플릿 (회사, 직무 정보 추가) ===
advice_prompt = PromptTemplate(
    input_variables=["분석결과", "회사", "직무"],
    template="""
아래 분석 결과와 지원 정보(지원 회사 및 직무)를 참고하여, 자소서 개선 및 보완을 위한 구체적인 조언을 작성해줘.
분석 결과:
--------------------------------------
{분석결과}
--------------------------------------
지원 정보:
- 지원하는 회사: {회사}
- 지원하는 직무: {직무}

조언은 실제 자소서에 적용 가능한 수정 방안과 개선 방향을 상세하게 제시하고, 지원 회사와 직무에 맞는 어필 포인트를 반영해줘.
"""
)

In [33]:
# === 자기소개서 예시 출력 프롬프트 템플릿 (회사, 직무 정보 포함) ===
example_prompt = PromptTemplate(
    input_variables=["조언결과", "회사", "직무"],
    template="""
아래 조언 결과를 바탕으로, 실제 지원자가 제출할 수 있는 자기소개서 예시를 작성해줘.
자기소개서는 다음과 같은 구조를 갖추어야 합니다:
1. **지원동기 및 비전:** 지원 동기, 본인의 비전, 그리고 지원 회사({회사}) 및 직무({직무})와의 시너지를 명확히 서술
2. **경험 및 역량:** 학습, 동아리/프로젝트 경험, 보유 기술 및 역량을 구체적으로 기술
3. **강점 및 보완 사항:** 분석된 강점을 부각시키고, 약점을 개선하기 위한 구체적인 실행 계획 제시
4. **향후 계획:** 입사 후 성장 방향과 목표

각 항목별로 소제목을 붙여 자기소개서 형식(각 섹션별로 구분)으로 작성해줘.
아래 조언 결과:
--------------------------------------
{조언결과}
--------------------------------------
자기소개서 예시:
"""
)

In [34]:
# 체인 생성: 분석 체인, 조언 체인, 예시 출력 체인
init_chain = LLMChain(llm=llm, prompt=init_prompt)
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
advice_chain = LLMChain(llm=llm, prompt=advice_prompt)
example_chain = LLMChain(llm=llm, prompt=example_prompt)

In [35]:
# === 사용자의 입력 ===
user_self_intro = """
안녕하세요, 저는 컴퓨터공학과를 전공하고 있는 학생입니다.
프로그래밍을 좋아하고, 다양한 프로젝트를 진행하며 실력을 키워왔습니다.
지원 회사인 가비아에서 소프트웨어 엔지니어로 일하고 싶습니다.

"""

# 추가 입력: 지원하는 회사와 직무
company = "가비아"
job_position = "소프트웨어 엔지니어"

In [36]:
# 0. 초기화 체인: 이전 프롬프트의 내용을 초기화하여 새로운 프롬프트 
init_result = init_chain.run(
    참고자료=combined_reference
)

In [37]:
# 1. 분석 체인 실행: 자소서 내용과 참고 자료, 지원 정보를 기반으로 강점/약점 및 적합성 평가
analysis_result = analysis_chain.run(
    자소서내용=user_self_intro,
    참고자료=combined_reference,
    회사=company,
    직무=job_position
)
print("=== 분석 결과 ===")
print(analysis_result)

=== 분석 결과 ===
### 예상 합격/불합격 여부: 불합격 / 점수: 5.5/10

#### 강점
1. **전공 관련성**: 컴퓨터공학과를 전공하고 있어 기본적인 이론과 기술을 학습한 점은 소프트웨어 엔지니어로서 긍정적인 요소입니다.
2. **프로그래밍에 대한 열정**: 프로그래밍에 대한 애정과 흥미를 보유하고 있어, 지속적으로 학습하고 성장할 가능성이 높습니다.
3. **프로젝트 경험**: 다양한 프로젝트를 진행해 보았다는 점은 실무에서 필요한 경험을 쌓았음을 시사합니다.

#### 약점
1. **구체성 부족**: '다양한 프로젝트'에 대한 구체적인 설명이 부족하여, 어떤 경험이 있는지 이해하기 어렵습니다.
2. **회사와의 연계 부족**: 지원하는 회사인 가비아에 대한 연구가 부족해 보이며, 동기 및 가치와의 일치성을 나타내는 부분이 결여되어 있습니다.
3. **경험의 깊이 부족**: 프로젝트에 대한 구체적인 역할이나 성과를 언급하지 않아 깊이 있는 경험을 전달하지 못하고 있습니다.

#### 적합성 평가
- **지원 동기 보완 필요**: 가비아에서 일하고 싶다는 의지와 이유를 좀 더 구체적으로 서술해야 합니다. 회사의 비전이나 최근 산업 동향과 연결하여 자신이 어떤 기여를 할 수 있을지 명확히 표현하는 것이 중요합니다.
- **관련 경험 확대**: 진행한 프로젝트의 세부사항, 자신이 맡은 역할 및 사용한 기술 등을 추가하여 실질적인 경험을 강조할 필요가 있습니다.
- **핵심 역량 명확화**: 소프트웨어 엔지니어로서 필요한 기술적 역량(예: 특정 프로그래밍 언어, 도구, 협업 툴 등)을 더 명확하게 제시하여 경쟁력을 높여야 합니다. 

위의 요소들을 개선하면 자소서의 완성도가 높아져 가비아와 소프트웨어 엔지니어 직무에 더 적합하게 보일 것입니다.


In [38]:
# 2. 조언 체인 실행: 분석 결과와 지원 정보를 기반으로 개선 조언 생성
advice_result = advice_chain.run(
    분석결과=analysis_result,
    회사=company,
    직무=job_position
)
print("\n=== 조언 결과 ===")
print(advice_result)


=== 조언 결과 ===
자기소개서를 보다 강력하게 만들기 위한 구체적인 방안과 개선 방향은 아래와 같습니다.

### 1. 구체적인 프로젝트 경험 서술

- **프로젝트 명 및 목표**: 자소서 시작 부분에 자신이 진행한 주요 프로젝트의 이름과 그 목표를 명확히 기재하세요. 예를 들어, "전공 수업의 일환으로 진행한 'X 프로젝트'는 A 문제를 해결하기 위해 설계되었습니다."와 같은 형태로 시작할 수 있습니다.

- **역할 및 기여도**: 각 프로젝트에서 자신의 역할과 기여를 명확히 서술하세요. "저는 백엔드 개발을 담당하여 RESTful API를 구현했습니다."와 같이 구체적인 기술과 역할을 강조해 보세요.

- **성과 및 학습 경험**: 프로젝트 결과를 수치나 구체적인 성과로 나타내세요. 예를 들어, "이 프로젝트를 통해 서비스 반응 속도를 30% 개선하는 데 기여했습니다." 또는 "특정 기술을 사용함으로써 문제 해결 능력을 향상시켰습니다." 같은 표현을 사용할 수 있습니다.

### 2. 회사와 연계된 지원 동기 제시

- **가비아 연구 및 비전**: 가비아가 어떤 비전을 가지고 있으며, 최근 어떤 산업 동향에 영향을 받고 있는지 연구하세요. 이를 바탕으로 "가비아의 AI 기반 솔루션 개발 계획에 매료되어 지원하게 되었습니다."와 같은 문장을 사용할 수 있습니다.

- **가치 일치**: 가비아의 핵심 가치나 미션과 자신의 가치관을 연결시키세요. "저는 혁신적인 기술을 통해 더 나은 사회를 만들고 싶습니다. 이 점에서 가비아가 추구하는 가치와 잘 맞다고 생각합니다." 같은 방식으로 명확히 설명합니다.

### 3. 핵심 기술 및 역량 강조

- **필요한 기술 명시**: 지원하는 소프트웨어 엔지니어 직무에 필요한 기술적 역량을 강조하세요. 예를 들어, "Java, Python 그리고 다양한 웹 프레임워크에 대한 이해를 가지고 있으며, 협업 도구인 Git과 Jira를 활용하여 팀 프로젝트를 효율적으로 진행한 경험이 있습니다." 같은 구체적인 내용을

In [ ]:
# 3. 예시 출력 체인 실행: 조언 결과와 지원 정보를 반영한 자기소개서 예시 생성
example_self_intro = example_chain.run(
    조언결과=advice_result,
    회사=company,
    직무=job_position
)
print("\n=== 자기소개서 예시 ===")
print(example_self_intro)